## Сбор данных

*Подключение библиотек для записи в файл и сбора данных*

In [100]:
import csv
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException, InvalidSessionIdException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

*Создание документа и заполнение первой строчки (названия столбцов) итогового файла*

In [101]:
with open("data/auto.csv", "a", newline="", encoding="cp1251") as file:
    writer = csv.writer(file, delimiter=",")
    writer.writerow([
        "model auto", "year of production", "mileage", "exterior colour", "tax",
        "transmission", "wheel drive", "condition", "number of owners", "engine volume",
        "engine power", "engine type", "fuel type", "acceleration", "MPG",
        "country of the car brand", "class", "top speed", "environmental class", "price"
    ])

*Создание списка с городами, по которым наш парсер будет проходиться*

In [102]:
CITIES = ["moskva", "sankt-peterburg", "vladimir"]

*Создание словаря, где ключ - это признак, с помощью которого мы будем искать данные на странице, а значение - название столбца, куда будет записываться значение, соответствующее признаку*

In [103]:
DICT_CHARACTERISTICS_MAIN = {
    "Год выпуска": "year of production",
    "Пробег": "mileage",
    "Цвет": "exterior colour",
    "Налог": "tax",
    "Коробка": "transmission",
    "Привод": "wheel drive",
    "Состояние": "condition",
    "Владельцы": "number of owners"
}

DICT_CHARACTERISTICS_OTHER = {
    "Объем": "engine volume",
    "Мощность": "engine power",
    "Тип двигателя": "engine type",
    "Топливо": "fuel type",
    "Разгон": "acceleration",
    "Расход": "MPG",
    "Страна марки": "country of the car brand",
    "Класс автомобиля": "class",
    "Максимальная скорость": "top speed",
    "Экологический класс": "environmental class"
}

*Объявляем переменную, которая будет означать общее количество страниц, которое нужно спарсить*

In [104]:
PAGES = 10

*Генерируем URL для страницы с объявлениями об автомобилях по указанному городу и номеру страницы*

In [105]:
def get_url(city, page):
    return f"https://auto.ru/{city}/cars/used/?page={page}"

*Получаем количество страниц с объявлениями об автомобилях в конкретном городе*

In [106]:
def get_pagination(driver):
    try:
        return int(driver.find_element(By.XPATH, "//a[contains(@class, 'ListingPagination__page')][last()]").get_attribute("href").split("page=")[1])
    except NoSuchElementException:
        return 1

*Извлекаем URL-адреса объявлений об автомобилях с текущей страницы и записываем их в список*

In [107]:
def get_urls_from_page(driver):
    array_auto = driver.find_elements(By.XPATH, "//a[@class='Link ListingItemTitle__link']")
    return [auto.get_attribute("href") for auto in array_auto]

*Проходимся поочерёдно по каждому городу, странице и сохраняем ссылки на объявления, вызываем вспомогательную функцию, которая найдёт все характеристики на странице, и записываем их в файл*

In [108]:
def processing_autos(driver, i_pages):
    for city in CITIES:
        driver.get(get_url(city, 1))
        pages = get_pagination(driver)
        for page in range(1, pages + 1):
            if (i_pages + 1 > PAGES):
                return
            i_pages += 1
            print(f"Page: {i_pages}")
            array_informations_auto = []
            driver.get(get_url(city, page))
            array_url_car = get_urls_from_page(driver)
            for url in array_url_car:
                auto_info = {}
                try:
                    driver.get(url)
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//h1[@class='CardHead__title']")))
                    processing_characteristics_auto(driver, auto_info)
                except TimeoutException:
                    pass
                array_informations_auto.append(auto_info)
                
            with open("data/auto.csv", 'a', newline='', encoding='cp1251') as file:
                writer = csv.writer(file, delimiter=',')
                for auto in array_informations_auto:
                    try:
                        writer.writerow([auto.get(column, '') for column in [
                            "model auto", "year of production", "mileage", "exterior colour", "tax", "transmission", 
                            "wheel drive", "condition", "number of owners", "engine volume", "engine power", "engine type", 
                            "fuel type", "acceleration", "MPG", "country of the car brand", "class", "top speed", 
                            "environmental class", "price"
                        ]])
                    except (KeyError, UnicodeEncodeError):
                        pass

*Вспомогательная функция*

In [109]:
def processing_characteristics_auto(driver, auto_info):
    try:
        model = driver.find_element(By.XPATH, "//h1[@class='CardHead__title']").text
        auto_info["model auto"] = model
    except NoSuchElementException:
        auto_info["model auto"] = "Unknown"

    try:
        price = driver.find_element(By.XPATH, "//span[@class='OfferPriceCaption__price']").text.replace('₽', '')
        auto_info["price"] = price
    except NoSuchElementException:
        auto_info["price"] = "Unknown"

    try:
        generation = driver.find_element(By.XPATH, "//span[contains(text(), 'Характеристики модели')]/../..")
        url = generation.get_attribute("href")
    except NoSuchElementException:
        url = None

    for key, value in DICT_CHARACTERISTICS_MAIN.items():
        try:
            pair = driver.find_element(By.XPATH, f"//div[contains(text(), '{key}')]/../div[@class='CardInfoRow__cell']/../div[2]")
            if key == "Налог":
                auto_info[value] = pair.text.replace('₽', '')
            else:
                auto_info[value] = pair.text
        except NoSuchElementException:
            auto_info[value] = "?"

    if url:
        driver.get(url)
        for key, value in DICT_CHARACTERISTICS_OTHER.items():
            try:
                pair = driver.find_element(By.XPATH, f"//span[contains(text(), '{key}')]/..//span[2]")
                auto_info[value] = pair.text
            except NoSuchElementException:
                auto_info[value] = "?"

*Отключаем обнаружение WebDriver, создаём экземпляр веб-драйвера Firefox и запускаем наш парсер*

In [110]:
options = webdriver.FirefoxOptions()
options.set_preference("dom.webdriver.enabled", False)
driver = webdriver.Firefox(options=options)

i_pages = 0
processing_autos(driver, i_pages)

driver.quit()

Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
